In [ ]:
!git clone https://huggingface.co/spaces/depth-anything/Depth-Anything-V2


fatal: destination path 'Depth-Anything-V2' already exists and is not an empty directory.


In [ ]:
%cd Depth-Anything-V2


/content/Depth-Anything-V2


In [ ]:
!pip install -r requirements.txt


In [ ]:
!pip install mediapipe opencv-python


In [ ]:
!pip install torch torchvision


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import torch
import math
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import files
import os

In [ ]:
# Initialize MediaPipe Pose for Pose Estimation
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

In [ ]:
# Import and Initialize Depth Anything V2 Model
from depth_anything_v2.dpt import DepthAnythingV2

depth_model = DepthAnythingV2(encoder='vitb', features=128, out_channels=[96, 192, 384, 768])
depth_model.load_state_dict(torch.load('/content/depth_anything_v2_vitb.pth', map_location='cpu'))
depth_model.eval()

<ipython-input-79-07be6a114ce3>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  depth_model.load_state_dict(torch.load('/content/depth_anything_v2_vitb.pth', map_location='

DepthAnythingV2(
  (pretrained): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-11): 12 x NestedTensorBlock(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
    

In [ ]:
# Upload your video for processing
from google.colab.patches import cv2_imshow
import cv2

video_path = '/content/Kirolos_video.mp4'

# Load the video
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Select frames at start, middle, and end (5 frames for CPU efficiency)
sampled_indices = [0, total_frames // 4, total_frames // 2, (3 * total_frames) // 4, total_frames - 1]
print(f"Processing frames at indices: {sampled_indices}")


Processing frames at indices: [0, 120, 240, 360, 480]


In [ ]:
# Initialize counters and lists
frame_counter = 0
processed_frames = []
foot_distances = []  # Store calculated distances in meters

# Process sampled frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print(f"End of video or failed to read frame at frame {frame_counter}.")
        break

    if frame_counter in sampled_indices:
        print(f"\nProcessing frame {frame_counter}...")

        # Convert frame to RGB for MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pose_results = pose.process(frame_rgb)

        if pose_results.pose_landmarks:
            landmarks = pose_results.pose_landmarks.landmark

            # Extract ankle keypoints
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]

            # Validate keypoint confidence scores
            if left_ankle.visibility < 0.5 or right_ankle.visibility < 0.5:
                print(f"Low confidence in ankle detection (Frame {frame_counter}). Skipping...")
                frame_counter += 1
                continue

            # Convert normalized coordinates to pixel values
            left_ankle_coords = np.array([left_ankle.x * frame.shape[1], left_ankle.y * frame.shape[0]])
            right_ankle_coords = np.array([right_ankle.x * frame.shape[1], right_ankle.y * frame.shape[0]])

            print(f"Left ankle: {left_ankle_coords}, Right ankle: {right_ankle_coords}")

            # Depth Estimation using Depth Anything V2
            depth_map = depth_model.infer_image(frame)

            # Resize depth map to match frame dimensions
            depth_map_resized = cv2.resize(depth_map, (frame.shape[1], frame.shape[0]))

            try:
                # Extract depth values from resized depth map
                z_left = depth_map_resized[int(left_ankle_coords[1]), int(left_ankle_coords[0])] / 1000  # Convert mm to meters
                z_right = depth_map_resized[int(right_ankle_coords[1]), int(right_ankle_coords[0])] / 1000  # Convert mm to meters

                # Validate depth values
                if z_left <= 0 or z_right <= 0:
                    print(f"Invalid depth values detected in frame {frame_counter}. Skipping...")
                    frame_counter += 1
                    continue

            except IndexError:
                print(f"Depth index out of bounds (Frame {frame_counter}). Skipping...")
                frame_counter += 1
                continue

            print(f"Raw Depth Values (in meters) - Left: {z_left}, Right: {z_right}")

            # FOV-Based Distance Calculation
            horizontal_pixel_distance = np.abs(left_ankle_coords[0] - right_ankle_coords[0])
            avg_depth = (z_left + z_right) / 2

            if avg_depth == 0:
                print(f"Zero depth detected (Frame {frame_counter}). Skipping...")
                continue

            # Calculate real-world distance using FOV and depth
            frame_width = frame.shape[1]
            real_world_distance = 2 * avg_depth * math.tan(HORIZONTAL_FOV_RADIANS / 2) * (horizontal_pixel_distance / frame_width)

            # Apply calibration based on real-world measurements
            real_world_distance *= calibration_constant

            # Smoothing over the last 3 frames
            window_size = 3
            if len(foot_distances) >= window_size:
                smoothed_distance = np.mean(foot_distances[-window_size:])
            else:
                smoothed_distance = real_world_distance

            print(f"FOV-Corrected Distance (Frame {frame_counter}): {smoothed_distance:.2f} m")

            foot_distances.append(smoothed_distance)

            # Annotate frame with distance
            cv2.putText(frame, f'Distance: {smoothed_distance:.2f} m', (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Draw ankle keypoints and connecting line
            cv2.circle(frame, tuple(left_ankle_coords.astype(int)), 5, (0, 255, 0), -1)
            cv2.circle(frame, tuple(right_ankle_coords.astype(int)), 5, (0, 255, 0), -1)
            cv2.line(frame, tuple(left_ankle_coords.astype(int)), tuple(right_ankle_coords.astype(int)), (255, 0, 0), 2)

            processed_frames.append(frame)

        else:
            print(f"No pose detected in frame {frame_counter}.")

    frame_counter += 1

# Release resources
cap.release()
try:
    pose.close()
except ValueError:
    print("Pose graph already closed.")



Processing frame 0...
Left ankle: [1211.03210449  198.14304113], Right ankle: [1195.84655762  509.66970921]
Raw Depth Values (in meters) - Left: 0.005888804912567138, Right: 0.006195943355560302
FOV-Corrected Distance (Frame 0): 0.01 m

Processing frame 120...
Left ankle: [877.21862793 276.3297987 ], Right ankle: [896.45950317 425.36826611]
Raw Depth Values (in meters) - Left: 0.003059448719024658, Right: 0.0036588859558105468
FOV-Corrected Distance (Frame 120): 0.01 m

Processing frame 240...
Left ankle: [763.96751404 306.78814888], Right ankle: [767.83363342 393.86089325]
Raw Depth Values (in meters) - Left: 0.0012249650955200196, Right: 0.0014204038381576537
FOV-Corrected Distance (Frame 240): 0.00 m

Processing frame 360...
Left ankle: [842.12944031 297.23650217], Right ankle: [819.45640564 422.82943726]
Raw Depth Values (in meters) - Left: 0.0023538775444030763, Right: 0.0023010103702545168
FOV-Corrected Distance (Frame 360): 0.01 m

Processing frame 480...
Left ankle: [1161.5938